<a href="https://colab.research.google.com/github/utkuyucel/Deep-Learning/blob/master/Keras_Cifar10_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from keras.utils import to_categorical
import numpy as np

(X_train, y_train),(X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [184]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

Found TPU at: grpc://10.80.194.98:8470


In [0]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

X_train /= 255
X_test /= 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [0]:
def train_input_fn(batch_size=1024):
  # Girişleri bir veri kümesine dönüştür. 
  dataset = tf.data.Dataset.from_tensor_slices((X_train,y_train))

  # Karıştır, tekrar et ve batch (küme) örnekleri 
  dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)

  # veri kümesine.
  return dataset

In [0]:
def test_input_fn(batch_size=1024):
  # Girişleri bir veri kümesine dönüştür. .
  dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))

  # sKarıştır, tekrar et ve batch (küme) örnekleri 
  dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)

  # veri kümesine.
  return dataset

In [0]:
def model(X_input):
  X_input = tf.keras.Input((32,32,3))
  X = tf.keras.layers.Conv2D(64, (3,3), strides = (1,1), padding = "same")(X_input)
  X = tf.keras.layers.Activation("relu")(X)
  X = tf.keras.layers.BatchNormalization()(X)
  
  
  
  
  X = tf.keras.layers.Flatten()(X)
  X = tf.keras.layers.Dense(128)(X)
  
  X = tf.keras.layers.Activation("relu")(X)
  
  pred = tf.keras.layers.Dense(10, activation = "softmax")(X)
  
  model = tf.keras.Model(inputs = X_input, outputs = pred)
  
  return model

In [0]:
model = model((32,32,3))

In [0]:
model.compile(optimizer = tf.train.AdamOptimizer(learning_rate = 0.001),
             loss = "categorical_crossentropy",
             metrics = ["acc"])

In [210]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

INFO:tensorflow:Querying Tensorflow master (grpc://10.80.194.98:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13170680794375988052)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 98274149842899398)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8698995895049676745)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1405650577223170611)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12018196556013156065)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/j

In [211]:
tpu_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_v1_6 (Ba (None, 32, 32, 64)        256       
_________________________________________________________________
flatten_6 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_89 (Dense)             (None, 128)               8388736   
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
__________

In [212]:
tpu_model.fit(
  train_input_fn,
  steps_per_epoch = 60,
  epochs=100,
)

Epoch 1/100
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(1024,), dtype=tf.int32, name=None), TensorSpec(shape=(1024, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(1024, 10), dtype=tf.float32, name=None)]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_40
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 6.568288326263428 secs
INFO:tensorflow:Setting weights on TPU model.
60/60 [==============================] - 18s 297ms/step - loss: 2.6512 - acc: 0.4639
Epoch 2/100
60/60 [==============================] - 6s 105ms/step - loss: 0.6064 - acc: 0.8051
Epoch 3/100
60/60 [==============================] - 6s 104ms/step - loss: 0.2379 - acc: 0.9427
Epoch 4/100
60/60 [==============================] - 6s 101ms/step - loss: 0.1092 - acc: 0.9790
Epoch 5/100
60/60 [==============================] - 6s 100ms/step - loss: 0.0462 - acc: 0.9941
Epoch 6/10

In [213]:
tpu_model.save_weights('./MNIST_TPU_1024.h5', overwrite=True)

INFO:tensorflow:Copying TPU weights to the CPU


In [214]:
tpu_model.evaluate(test_input_fn, steps = 100)

INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(1024,), dtype=tf.int32, name=None), TensorSpec(shape=(1024, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(1024, 10), dtype=tf.float32, name=None)]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_40
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 4.889551401138306 secs
100/100 [==============================] - 17s 170ms/step - loss: 3.2221 - acc: 0.6096


[3.222080225944519, 0.60962033]